In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from glob import glob
import ast
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageOps
import cv2

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Colab Notebooks/Doodle Sketch recognition"
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

In [ ]:
!kaggle competitions download -c quickdraw-doodle-recognition

In [ ]:
#unzipping the zip files and deleting the zip files
!unzip \*.zip  && rm *.zip

In [ ]:
train_dir = '/content/'

**Let's see how the training data looks like**

In [ ]:
train = pd.DataFrame()

# how many rows from each csv?
get_n = 1

for file in os.listdir(train_dir):
  if file.endswith(".csv"):
    train = train.append(pd.read_csv(train_dir + file, index_col='key_id', nrows=10000))

In [ ]:
train.head()

In [ ]:
train.isna().any()

In [ ]:
train = train.dropna()

**Let's drop the timestamp, countrycode**

In [ ]:
del train['timestamp']
del train['countrycode']
del train['recognized']

**Let's convert the drawing to np array**

In [ ]:
def draw_it(strokes):
    # PIL.Image.new() method creates a new image with the given mode and size. 
    image = Image.new("P", (256,256), color=255)
    # The ImageDraw module provides simple 2D graphics for Image objects.
    # You can use this module to create new images, annotate or retouch existing images, 
    # and to generate graphics on the fly for web use.
    image_draw = ImageDraw.Draw(image)
    
    # draw strokes on image
    for stroke in ast.literal_eval(strokes):
        for i in range(len(stroke[0])-1):
            image_draw.line([stroke[0][i], 
                             stroke[1][i],
                             stroke[0][i+1], 
                             stroke[1][i+1]],
                            fill=0, width=5)
    # resize the image
    image = image.resize((64, 64))/255
    image = np.array(image).reshape((64, 64, 1))
   
    return image

**Create feature with np array of the drawing**

In [ ]:
train['drawing_np'] = train['drawing'].apply(draw_it)

In [ ]:
train.head()

**Let's plot a image and see**

In [ ]:
def plot_np_image(image_arr, word):
    """ Method to plot np array image """
    
    fig = plt.figure()
    ax2 = fig.add_subplot(111, frameon=False)
    ax2.imshow(image_arr[:,:,0])
    plt.title(word)
    plt.show()
    print('Image shape: {}'.format(np.array(image_arr).shape))

plot_np_image(train['drawing_np'].iloc[0], train['word'].iloc[0])


**Let's shuffle the dataset**

In [ ]:
train = train.sample(frac=1).reset_index(drop=True)

In [ ]:
train.head()

In [ ]:
del train['drawing']

**Independant Dependant Var**

In [ ]:
from tensorflow.keras import utils
from sklearn.preprocessing import LabelEncoder

num_classes = len(train['word'].unique())

In [ ]:
x = np.stack(train['drawing_np'].to_numpy())
x.shape

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
y = train['word']
y = label_encoder.fit_transform(y)
y_lbl = y
y = utils.to_categorical(y, num_classes)

In [ ]:
label_encoder.inverse_transform(y_lbl)

**Train test split**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.metrics import top_k_categorical_accuracy
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(64, 64, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(680, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(np.array(x_train), np.array(y_train), batch_size=32, epochs=10)

In [ ]:
preds = model.predict(x_test)

In [ ]:
_, acc = model.evaluate(x_test, y_test, verbose=0)

In [ ]:
acc

In [ ]:
test = pd.read_csv('/content/test_simplified.csv')

In [ ]:
test['drawing'] = test['drawing'].apply(draw_it)

In [ ]:
sample_file = test['drawing'].iloc[0]
plot_np_image(sample_file, 'test')

In [ ]:
sample_np = np.stack(sample_file)

In [ ]:
sample_file = sample_file.reshape((1,) + sample_file.shape)

In [ ]:
sample_file.shape

In [ ]:
test_arr = np.stack(test['drawing'].to_numpy())

In [ ]:
test_arr.shape

In [ ]:
def get_predictions_for_one(new_img):
  pred = model.predict(new_img)
  pred =  np.argsort(-pred)[:, 0:3]
  print(pred)
  return label_encoder.inverse_transform(pred[0])


In [ ]:
print(get_predictions_for_one(ze))

In [ ]:
preds = model.predict(test_arr)

In [ ]:
n =  np.argsort(-preds)[:, 0:3]

In [ ]:
top_3 = []
for i in n:
  top_3.append(label_encoder.inverse_transform(i))

In [ ]:
pred_df = pd.DataFrame(top_3, columns=['pred_1', 'pred_2', 'pred_3'])

In [ ]:
pred_df.shape

In [ ]:
pred_df = pred_df.replace(' ', '_', regex=True)

In [ ]:
pred_df['word'] = pred_df[pred_df.columns].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

In [ ]:
pred_df.shape

In [ ]:
pred_df.head(50)

In [ ]:
test['word'] = pred_df['word']

In [ ]:
del test['countrycode']

In [ ]:
del test['drawing']

In [ ]:
test.head(10)

In [ ]:
test.to_csv('my_preds.csv', index=False)